In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import optuna
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Attention, Concatenate, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
df = pd.read_csv('df_interpolated (1).csv')  # Update with your dataset path
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Select relevant columns, exclude 'date' for scaling
columns = ['gold_price_usd', 'silver_price', 's&p_500_index', 'nyse_com_index', 'usd_selling_exrate', 'gold_lkr']
data = df[columns].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define window size for LSTM
window_size = 30

# Create sequences
X, y = [], []
for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i-window_size:i, :])
    y.append(scaled_data[i, -1])  # 'gold_lkr' is the last column

X, y = np.array(X), np.array(y)

# Split data into train and test sets
split = int(0.8 * len(data))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

# Define the model with different architectures
def create_model(trial):
    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
    
    # Choose the number of layers and units
    n_layers = trial.suggest_int('n_layers', 1, 4)
    units = trial.suggest_int('units', 50, 200)
    
    # Choose architecture type: LSTM, GRU, or Bidirectional LSTM
    rnn_type = trial.suggest_categorical('rnn_type', ['LSTM', 'GRU', 'Bidirectional LSTM'])
    
    if rnn_type == 'LSTM':
        rnn_out = LSTM(units=units, return_sequences=True)(input_layer)
    elif rnn_type == 'GRU':
        rnn_out = GRU(units=units, return_sequences=True)(input_layer)
    else:
        rnn_out = Bidirectional(LSTM(units=units, return_sequences=True))(input_layer)

    rnn_out = Dropout(0.2)(rnn_out)
    
    for _ in range(n_layers - 1):
        if rnn_type == 'LSTM':
            rnn_out = LSTM(units=units, return_sequences=True)(rnn_out)
        elif rnn_type == 'GRU':
            rnn_out = GRU(units=units, return_sequences=True)(rnn_out)
        else:
            rnn_out = Bidirectional(LSTM(units=units, return_sequences=True))(rnn_out)
        rnn_out = Dropout(0.2)(rnn_out)
    
    # Attention layer
    attention = Attention()([rnn_out, rnn_out])
    concat = Concatenate()([rnn_out, attention])
    
    # Final LSTM/GRU layer
    if rnn_type == 'LSTM':
        rnn_out = LSTM(units=units)(concat)
    elif rnn_type == 'GRU':
        rnn_out = GRU(units=units)(concat)
    else:
        rnn_out = Bidirectional(LSTM(units=units))(concat)

    rnn_out = Dropout(0.2)(rnn_out)
    output_layer = Dense(units=1)(rnn_out)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

# Objective function for Optuna
def objective(trial):
    model = create_model(trial)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=0, callbacks=[early_stopping])
    
    val_loss = min(history.history['val_loss'])
    
    return val_loss

# Optimize hyperparameters using Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print(f'Best parameters: {study.best_params}')

# Build and train the final model with the best parameters
best_params = study.best_params
model = create_model(study.best_trial)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)

# Make predictions
predictions = model.predict(X_test)

# Inverse scaling for predictions and actual values
X_test_inverse = np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1)
predictions = np.concatenate((X_test[:, -1, :-1], predictions), axis=1)
actual_values = scaler.inverse_transform(X_test_inverse)[:, -1]
predictions = scaler.inverse_transform(predictions)[:, -1]

# Calculate performance metrics
mse = mean_squared_error(actual_values, predictions)
print(f'Mean Squared Error: {mse}')

# Generate future dates
last_date = df['date'].iloc[-1]
future_dates = pd.date_range(start=last_date, periods=30, freq='B')  # Business days for 30 days

# Prepare data for future predictions
future_predictions = []
last_sequence = scaled_data[-window_size:]

for date in future_dates:
    prediction = model.predict(last_sequence[np.newaxis, :, :])[0, 0]
    future_predictions.append(prediction)
    new_sequence = np.append(last_sequence[1:], np.expand_dims(np.append(last_sequence[-1, :-1], prediction), axis=0), axis=0)
    last_sequence = new_sequence

# Inverse scaling for future predictions
future_predictions = scaler.inverse_transform(np.concatenate((np.tile(scaled_data[-1, :-1], (len(future_predictions), 1)), np.array(future_predictions).reshape(-1, 1)), axis=1))[:, -1]

# Plot results
plt.figure(figsize=(14, 7))
plt.plot(df['date'][split+window_size:], actual_values, label='Actual Gold Price')
plt.plot(df['date'][split+window_size:], predictions, label='Predicted Gold Price')
plt.plot(future_dates, future_predictions, label='Future Predictions')
plt.xlabel('Date')
plt.ylabel('Gold Price (LKR)')
plt.title('Gold Price Prediction')
plt.legend()
plt.show()


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-09-13 03:51:00,088] A new study created in memory with name: no-name-0dff7ec8-4f9c-4ba5-a027-cbe5ea925e1f
[I 2024-09-13 03:51:59,174] Trial 0 finished with value: 0.011119917035102844 and parameters: {'n_layers': 4, 'units': 89, 'rnn_type': 'GRU'}. Best is trial 0 with value: 0.011119917035102844.
[I 2024-09-13 04:05:16,580] Trial 1 finished with value: 0.011439342983067036 and parameters: {'n_layers': 4, 'units': 148, 'rnn_type': 'Bidirectional LSTM'}. Best is trial 0 with value: 0.011119917035102844.
[I 2024-09-13 04:06:01,916] Trial 2 finished with value: 0.0074208881705999374 and parameters: {'n_layers': 4, 'units': 51, 'rnn_type': 'GRU'}. Best is trial 2 with value: 0.0074208881705999374.
[I 2024-09-13 04:07:12,

KeyboardInterrupt: 